In [1]:
import torch
import numpy as np
import glob
from torchvision import transforms
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import argparse

In [9]:
class ContrasiveDataset(Dataset):
    def __init__(self,train_folder,image_size = 280):
        self.train_folder =  train_folder
        self.data = glob.glob(train_folder)
        self.transform = transforms.Compose([
            transforms.Resize((image_size, image_size)),
            transforms.ToTensor(),
        ])
        self.grey_transform = transforms.Compose([
            transforms.Resize((image_size, image_size)),
            transforms.ToTensor(),
            transforms.Lambda(lambda x: torch.cat([x, x, x], 0))
        ])
    def __getitem__(self, index):
        '''
        Accept pid for the identity
        Return all the image of of it
        '''
        return self.load_img(self.data[index])
    
    def load_img(self,path):
        with open(path,"rb") as f:
            img = Image.open(f)
            img.load()
            grey= Image.open(f).convert('L')
            grey.load()
        return self.transform(img), self.grey_transform(grey)
            
    def __len__(self):
        return len(self.data)
    
    
def get_train_loader(path, **kwargs):
    dataset = ContrasiveDataset(path,**kwargs)
    return DataLoader(dataset,4)

In [10]:
path = "../dataset/market1501/Market1501/bounding_box_train/*.jpg"

In [11]:
train_loader = get_train_loader(path)

In [12]:
def train_constraive_learning(model, train_loader, args):
    optimizer = torch.optim.Adam(lr=args.lr, weight_decay=args.weight_decay)
    

In [13]:
def main():
    args = parser.parse_args()
    model = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14') # replace by your saved model
    train_loader = get_train_loader(args.dataset)
    # path = "../dataset/market1501/Market1501/bounding_box_train/*.jpg"
    train_constraive_learning(model, train_loader, args)

In [14]:
sample = next(iter(train_loader))

In [15]:
color = sample[0]
grey =sample[1]

In [18]:
model = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')

Using cache found in /Users/daikeig/.cache/torch/hub/facebookresearch_dinov2_main
xFormers not available
xFormers not available


In [19]:
loss = torch.nn.CrossEntropyLoss()

In [20]:
color_embedding = model(color)

In [23]:
grey_embedding = model(grey)

In [24]:
color_embedding

tensor([[-1.3704,  2.3580,  2.1882,  ..., -2.0469,  1.5106,  0.4109],
        [-4.8266, -0.2898,  1.5311,  ..., -0.3666,  1.4601,  0.5469],
        [-5.6139,  2.1885,  4.0994,  ..., -2.6303,  1.5543, -0.8897],
        [-4.1618,  1.7416, -0.4323,  ..., -0.8462, -0.4044,  1.9370]],
       grad_fn=<SelectBackward0>)

In [25]:
grey_embedding

tensor([[-1.0384,  2.5348,  2.0379,  ..., -2.0586,  0.7261,  0.1235],
        [-4.1456,  0.2656,  1.7703,  ..., -0.7090, -0.3961, -0.1105],
        [-5.4135,  2.0006,  3.0065,  ..., -2.4226,  2.1922, -0.8414],
        [-1.9188,  2.2350, -0.4709,  ..., -0.8757, -1.3990,  1.6791]],
       grad_fn=<SelectBackward0>)

In [26]:
loss(color_embedding,grey_embedding)

tensor(-1968.2583, grad_fn=<DivBackward1>)

In [27]:
from info_nce import InfoNCE